# This file separates the dataset into multiple groups to calculate their average scores in different qualities.

In [ ]:
import pandas as pd

file_path = '../results/rationales_analysis_scores.xlsx'
sheets_dict = pd.read_excel(file_path, sheet_name=None)

sheets_dict

In [ ]:
# Initialize a dictionary to store correlation results
correlations = {}

# Iterate through each sheet
for sheet_name, df in sheets_dict.items():
    # Ensure the required columns exist
    if 'is_correct' in df.columns and 'visual_fidelity' in df.columns:
        # Compute the Pearson correlation
        corr = df['is_correct'].corr(df['visual_fidelity'])
        correlations[sheet_name] = corr
        print(f"Correlation for sheet {sheet_name}: {corr}")
    else:
        # Handle cases where columns might be missing
        correlations[sheet_name] = None
        print(f"Columns missing in sheet: {sheet_name}")

print("Mean correlation:", sum(correlations.values()) / len(correlations))


# List to hold DataFrames that have the required columns
valid_dfs = []

# Iterate through each sheet and collect valid DataFrames
for sheet_name, df in sheets_dict.items():
    if 'is_correct' in df.columns and 'visual_fidelity' in df.columns:
        # Select only the relevant columns
        valid_dfs.append(df[['is_correct', 'visual_fidelity']])
    else:
        print(f"Skipping sheet '{sheet_name}' due to missing columns.")

# Check if there are valid DataFrames to concatenate
if valid_dfs:
    # Concatenate all valid DataFrames
    combined_df = pd.concat(valid_dfs, ignore_index=True)
    print(f"Total records after concatenation: {combined_df.shape[0]}")
else:
    print("No valid DataFrames found to concatenate.")
    combined_df = None
    
if combined_df is not None:
    # Compute the Pearson correlation on the combined data
    overall_corr = combined_df['is_correct'].corr(combined_df['visual_fidelity'])
    print(f"Overall correlation between 'is_correct' and 'Visual Fidelity' across all sheets: {overall_corr}")

print()


In [ ]:
import pandas as pd

# Adjust pandas settings
pd.set_option('display.width', 1000)  # Increase display width
pd.set_option('display.max_colwidth', None)  # Don't wrap text in columns
pd.set_option('display.max_columns', None)  # Display all columns


for sheet_name, df in sheets_dict.items():
    print(sheet_name)
    
    # Compute averages grouped by 'is_correct'
    averages = df.groupby('is_correct')[
        'support', 
        'informativeness', 
        'commonsense_plausibility', 
        'strict_sim', 
        'visual_fidelity'
    ].mean()

    # Count instances in each group
    counts = df['is_correct'].value_counts()

    # Print in the required order
    print("Averages (grouped by is_correct):")
    print(averages.reindex([1, 0]))  # Order vertically by 1 (correct) and 0 (incorrect)
    print("\nCounts (grouped by is_correct):")
    print(counts.reindex([1, 0], fill_value=0))  # Ensure the order 1, 0 even if one group is missing
    print('*' * 100)

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Define the path to your Excel file
file_path = '../results/rationales_analysis_scores.xlsx'

# Load all sheets into a dictionary of DataFrames
sheets_dict = pd.read_excel(file_path, sheet_name=None)

# 1. Compute Per-Sheet Correlations
correlations = {}

for sheet_name, df in sheets_dict.items():
    # Compute the Pearson correlation between 'is_correct' and 'visual_fidelity'
    corr = df['is_correct'].corr(df['visual_fidelity'])
    correlations[sheet_name] = corr
    print(f"Correlation for sheet '{sheet_name}': {corr}")

# Calculate the mean correlation across all sheets
mean_correlation = sum(correlations.values()) / len(correlations)
print(f"\nMean correlation across all sheets: {mean_correlation:.2f}")

# 2. Concatenate All Data and Compute Overall Correlation
# Select only the relevant columns and concatenate
combined_df = pd.concat([df[['is_correct', 'visual_fidelity']] for df in sheets_dict.values()], ignore_index=True)
print(f"\nTotal records after concatenation: {combined_df.shape[0]}")

# Compute the overall Pearson correlation
overall_corr = combined_df['is_correct'].corr(combined_df['visual_fidelity'])
print(f"Overall correlation between 'is_correct' and 'Visual Fidelity' across all sheets: {overall_corr:.2f}")

# 3. Plotting the Correlation Heatmap
plt.figure(figsize=(6, 4))
sns.heatmap(combined_df[['is_correct', 'visual_fidelity']].corr(), annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Heatmap')
plt.show()



# 4. Analyzing Accurate Instances (is_correct == 1)
# Filter rows where 'is_correct' is 1
combined_accurate_df = combined_df[combined_df['is_correct'] == 1]
print(f"\nTotal accurate records: {combined_accurate_df.shape[0]}")

if combined_accurate_df is not None:
    # Compute the Pearson correlation on the combined data
    overall_corr = combined_accurate_df['is_correct'].corr(combined_accurate_df['visual_fidelity'])
    print(f"Overall correlation between 'is_correct' and 'Visual Fidelity' across all sheets: {overall_corr}")

# **Note**: Computing correlation between 'is_correct' and 'visual_fidelity' within accurate instances is meaningless
# because 'is_correct' is constant (always 1) in this subset.

# Instead, let's analyze the distribution of 'visual_fidelity' among accurate instances
plt.figure(figsize=(6, 4))
sns.countplot(x='visual_fidelity', data=combined_accurate_df)
plt.title('Distribution of Visual Fidelity for Accurate Instances')
plt.xlabel('Visual Fidelity')
plt.ylabel('Count')
plt.show()


In [ ]:
combined_accurate_df

In [ ]:
for sheet_name, df in sheets_dict.items():
    print(sheet_name)

    # Compute averages grouped by 'is_correct'
    averages = df.groupby('is_correct')[
        'support', 
        'informativeness', 
        'commonsense_plausibility', 
        'strict_sim', 
        'visual_fidelity'
    ].mean()

    # Calculate percentage drops from is_correct=1 to is_correct=0
    drop_percentages = (averages.loc[1] - averages.loc[0]) / averages.loc[1] * 100

    # Highlight the last two columns
    significant_drops = drop_percentages[['strict_sim', 'visual_fidelity']]
    other_drops = drop_percentages.drop(['strict_sim', 'visual_fidelity'])

    print("Drop Percentages:")
    print(drop_percentages)
    print("\nSignificant Drops (strict_sim and visual_fidelity):")
    print(significant_drops)
    print("\nOther Metric Drops:")
    print(other_drops)
    print('*' * 100)


In [ ]:
import pandas as pd
import ast

# Read the Excel file with all sheets
file_path = '../results/rationales_analysis_full_details.xlsx'
sheets_dict = pd.read_excel(file_path, sheet_name=None)

def parse_column(column):
    """
    Parse the string representations of lists into actual lists.
    """
    return column.apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

def convert_yes_no(series):
    """
    Convert 'Yes' to 1 and 'No' to 0 in each sublist.
    """
    def convert_list(lst):
        return [1 if isinstance(item, str) and item.strip().lower() == 'yes' 
                else 0 if isinstance(item, str) and item.strip().lower() == 'no' 
                else None for item in lst]
    
    return series.apply(convert_list)

def calculate_micro_average(series):
    """
    Calculate the micro average: sum of all elements divided by the total number of elements.
    """
    flattened = [item for sublist in series for item in sublist if item is not None]
    return sum(flattened) / len(flattened) if flattened else 0

def calculate_macro_average(series):
    """
    Calculate the macro average: average of the mean of each sublist.
    """
    means = [sum(sublist) / len(sublist) if sublist else 0 for sublist in series]
    return sum(means) / len(means) if means else 0

summary = []

for sheet_name, df in sheets_dict.items():
    if 'vf_answers_GPT_converted' not in df.columns:
        print(f"Sheet '{sheet_name}' does not contain 'vf_answers_GPT_converted' column.")
        continue
    
    if 'is_correct' not in df.columns:
        print(f"Sheet '{sheet_name}' does not contain 'is_correct' column. Skipping additional grouping analysis.")
        # Optionally, you can choose to process the sheet without grouping
        # For now, we'll skip additional grouping if 'is_correct' is missing
        process_grouping = False
    else:
        process_grouping = True

    # Parse the column
    df['parsed_scores'] = parse_column(df['vf_answers_GPT_converted'])
    
    # Ensure that all parsed entries are lists
    if not df['parsed_scores'].apply(lambda x: isinstance(x, list)).all():
        print(f"Sheet '{sheet_name}' has non-list entries in 'vf_answers_GPT_converted'. Skipping.")
        continue
    
    # Convert 'Yes'/'No' to 1/0
    df['numeric_scores'] = convert_yes_no(df['parsed_scores'])
    
    # Check for None values indicating unexpected entries
    if df['numeric_scores'].apply(lambda x: any(item is None for item in x)).any():
        print(f"Sheet '{sheet_name}' contains entries other than 'Yes'/'No'. These will be treated as 0.")
        # Optionally, you can handle these cases differently
        df['numeric_scores'] = df['numeric_scores'].apply(lambda lst: [item if item is not None else 0 for item in lst])
    
    # Calculate overall micro and macro averages
    micro_avg = calculate_micro_average(df['numeric_scores'])
    macro_avg = calculate_macro_average(df['numeric_scores'])
    
    # Initialize summary entry for the sheet
    summary_entry = {
        'Sheet': sheet_name,
        'Group': 'All',
        'Micro Average': micro_avg,
        'Macro Average': macro_avg
    }
    summary.append(summary_entry)
    
    # If grouping by 'is_correct' is applicable
    if process_grouping:
        grouped = df.groupby('is_correct')
        for group_value, group_df in grouped:
            group_label = 'Correct' if group_value == 1 else 'Incorrect'
            
            # Calculate micro and macro averages within the group
            group_micro_avg = calculate_micro_average(group_df['numeric_scores'])
            group_macro_avg = calculate_macro_average(group_df['numeric_scores'])
            
            # Append to summary
            summary.append({
                'Sheet': sheet_name,
                'Group': group_label,
                'Micro Average': group_micro_avg,
                'Macro Average': group_macro_avg
            })

# Create a summary DataFrame
summary_df = pd.DataFrame(summary)

# Optional: Order the summary for better readability
summary_df = summary_df.sort_values(by=['Sheet', 'Group']).reset_index(drop=True)

# Display the summary
print(summary_df)